## Assignment03: SMILE Experiment
### Sam Remmey, sqr8ap
#### Collaborated with Sara Grace Kennedy (sgk9hr) and used code from the lessons

In [14]:
import random
import csv
from copy import deepcopy
import pickle
import os
import pandas as pd
from smile.common import *

In [15]:
os.chdir('/Users/Samantha/Desktop/PSYC4215')

In [16]:
def read_and_shuffle(pool_file):
    """Read in and shuffle a pool."""
    # create a dictionary reader
    dr = csv.DictReader(open(pool_file, 'r'))

    # read in all the lines into a list of dicts
    pool = [l for l in dr]

    # shuffle it so that the we get new items each time
    random.shuffle(pool)
    
    # report out some pool info
    print(pool_file, len(pool))

    # return the shuffled pool
    return pool

### Configuration

In [17]:
pool_files = {'pos': 'pos_pool.csv',
              'neg': 'neg_pool.csv',
              'neu': 'neu_pool.csv'}

rep_conds = ['once', 'repeat', 'refresh']
val_conds = ['pos', 'neg', 'neu']

# what to show for refreshed items
ref_text = '+'

num_reps = 1
num_blocks = 2
display_time = 1.5 # how long each word will be displayed for in the study trials
font_size = 50
resp_keys = ['F','J']
ISI_dur = 0.5 # inter-stimulus duration
ISI_jitter = 0.5


### List Generation

In [18]:
# read all the pools into a dictionary
pools = {val: read_and_shuffle(pool_files[val])
         for val in val_conds}

# show the first 5 items of the pos pool
pools['pos'][:5]

pos_pool.csv 301
neg_pool.csv 292
neu_pool.csv 208


[{'description': 'travel',
  'word_no': '1018',
  'valence_mean': '7.0999999999999996',
  'valence_sd': '2.0',
  'arousal_mean': '6.21',
  'arousal_sd': '2.5099999999999998',
  'dominance_mean': '6.3099999999999996',
  'dominance_sd': '2.0800000000000001',
  'word_frequency': '61'},
 {'description': 'waterfall',
  'word_no': '487',
  'valence_mean': '7.8799999999999999',
  'valence_sd': '1.03',
  'arousal_mean': '5.3700000000000001',
  'arousal_sd': '2.8399999999999999',
  'dominance_mean': '5.2000000000000002',
  'dominance_sd': '2.1800000000000002',
  'word_frequency': '2'},
 {'description': 'youth',
  'word_no': '1039',
  'valence_mean': '6.75',
  'valence_sd': '2.29',
  'arousal_mean': '5.6699999999999999',
  'arousal_sd': '2.52',
  'dominance_mean': '5.1100000000000003',
  'dominance_sd': '2.5499999999999998',
  'word_frequency': '82'},
 {'description': 'awed',
  'word_no': '30',
  'valence_mean': '6.7000000000000002',
  'valence_sd': '1.3799999999999999',
  'arousal_mean': '5.740

In [19]:
# create the conds
# fully crossed with all combos of val and rep
conds = []
for val in val_conds:
    for rep in rep_conds:
        # I decided to call the repetition condition cond
        conds.append({'valence': val, 'cond': rep})
conds

[{'valence': 'pos', 'cond': 'once'},
 {'valence': 'pos', 'cond': 'repeat'},
 {'valence': 'pos', 'cond': 'refresh'},
 {'valence': 'neg', 'cond': 'once'},
 {'valence': 'neg', 'cond': 'repeat'},
 {'valence': 'neg', 'cond': 'refresh'},
 {'valence': 'neu', 'cond': 'once'},
 {'valence': 'neu', 'cond': 'repeat'},
 {'valence': 'neu', 'cond': 'refresh'}]

In [20]:
# make a function for generating a block
# with a study and test list
def make_block():
    """Generate a block, uses global variables"""
    # loop and create the repeated conditions
    block_conds = []
    for i in range(num_reps):
        # extend the trials with copies of the conditions
        block_conds.extend(deepcopy(conds))

    # shuffle the conds for that block
    random.shuffle(block_conds)

    # loop over block conds and add items to study/test lists
    study_list = []
    test_list = []
    for cond in block_conds:
        # use the valence to grab study and test items
        study_item = pools[cond['valence']].pop()
        test_item = pools[cond['valence']].pop()

        # update with the cond info
        study_item.update(cond)
        test_item.update(cond)

        # add in relevant info for study and test
        study_item['pres_num'] = 1
        study_item['type'] = 'target'
        test_item['type'] = 'lure'
        test_item['pres_num'] = 1   # just so the keys match

        # append them to the respective lists
        # study item is added to both study and test
        study_list.append(study_item)
        test_list.append(study_item)
        test_list.append(test_item)

        # process the repetition and refresh conditions
        if cond['cond'] in ['refresh', 'repeat']:
            # copy the study item
            rep_item = deepcopy(study_item)

            # modify required values
            rep_item['pres_num'] = 2

            # change the description if refreshing
            if cond['cond'] == 'refresh':
                rep_item['description'] = ref_text

            # append it to the study list
            study_list.append(rep_item)
            
    # must shuffle the test list
    random.shuffle(test_list)
    
    # make a dictionary to return
    block = {'study': study_list, 'test': test_list}
    
    return block

# generate the proper number of blocks
blocks = []
for b in range(num_blocks):
    blocks.append(make_block())


### SMILE Experiment

In [21]:
exp = Experiment(name='MEMORY', show_splash=False, fullscreen=False, debug=True)

Wait(1.0)

Label(text = f"During this task you will first be presented with a series of words that you should study carefully. \nYou will then be presented with another series of words, at which point you will be asked \nto indicate whether each word is new- i.e., you have not seen it before, or old- i.e, you saw it in the previous series of words. \nPress the F key for old and the J key for new. When presented with a \'{ref_text}\' symbol, you should refresh your memory\n of the previous item.", 
      duration = 15.0, font_size = 25)
Wait(1.0)
Label(text = 'Let\'s begin!',duration=2.0)

Wait(0.5)

with Loop(blocks) as block:
    Label(text = 'Press any key to begin the next block.')
    with UntilDone():
        KeyPress()
    Wait(ISI_dur, jitter=ISI_jitter) # waiting before beginning block

    # study list:
    with Loop(block.current['study']) as trial: 
        stim = Label(text=trial.current['description'],
                     font_size=font_size, duration=display_time)

        Wait(ISI_dur, jitter=ISI_jitter)

        Log(trial.current, name='study_memory',
           stim_on=stim.appear_time)

    # reminding participant which keys to press:
    Label(text = 'Now you will be presented with the test list. \nPress F if you have seen the word before and J if you have not.\nPress any key to begin.',)
    with UntilDone():
        KeyPress()
    Wait(ISI_dur, jitter=ISI_jitter)

    # test list:
    with Loop(block.current['test']) as trial:
        stim = Label(text=trial.current['description'],
                     font_size=font_size)
        with UntilDone():
            kp = KeyPress(keys=resp_keys)

        Wait(ISI_dur, jitter=ISI_jitter)

        Log(trial.current, name='test_memory',
            stim_on=stim.appear_time,
            resp=kp.pressed,
            resp_time=kp.press_time)
    
# telling participant they're done
Label(text = 'Congrats! You\'re done!', duration = 2.0)


    

<LabelBuilder file='/var/folders/13/9xyr4p8x2kq9msv1m8ysrjq00000gn/T/ipykernel_67434/4199571324.py', line=49, name=None, id=15af0d850>

In [22]:
exp.run()

In [3]:
# reading in some test data from the logs

from smile.log import log2dl
dl = log2dl('/Users/Samantha/Desktop/PSYC4215/data/MEMORY/test000/20231005_204948/log_test_memory_0.slog')

In [4]:
dl[:5]

[{'stim_on_time': 261919.471507541,
  'stim_on_error': 0.0,
  'resp': 'J',
  'resp_time_time': 261920.6861710415,
  'resp_time_error': 0.0017912085022544488,
  'log_time': 261921.43553788672,
  'description': 'joke',
  'word_no': '826',
  'valence_mean': '8.0999999999999996',
  'valence_sd': '1.3600000000000001',
  'arousal_mean': '6.7400000000000002',
  'arousal_sd': '1.8400000000000001',
  'dominance_mean': '6.1500000000000004',
  'dominance_sd': '1.8600000000000001',
  'word_frequency': '22',
  'valence': 'pos',
  'cond': 'refresh',
  'type': 'lure',
  'pres_num': 1,
  'log_num': 0},
 {'stim_on_time': 261921.437863708,
  'stim_on_error': 0.0,
  'resp': 'F',
  'resp_time_time': 261922.2763121245,
  'resp_time_error': 0.00016245849838014692,
  'log_time': 261923.0344231907,
  'description': 'death',
  'word_no': '100',
  'valence_mean': '1.6100000000000001',
  'valence_sd': '1.3999999999999999',
  'arousal_mean': '4.5899999999999999',
  'arousal_sd': '3.0699999999999998',
  'dominance